In [1]:
import requests
import pandas as pd

# List of ISO-3 codes for your 39 countries
countries = [
    "ZAF", "ARG", "AZE", "BGD", "BRA", "CHL", "CHN", "COL", "CIV",
    "EGY", "ECU", "ETH", "GEO", "GHA", "IND", "IDN", "JOR", "KAZ",
    "KEN", "MYS", "MAR", "MEX", "NGA", "PAN", "PAK", "PER", "PHL",
    "POL", "DOM", "ROU", "SEN", "SRB", "LKA", "THA", "TUN", "TUR",
    "UKR", "URY", "VNM"
]

def fetch_indicator_data(indicator_code, indicator_name):
    country_str = ";".join(countries)
    url = f"https://api.worldbank.org/v2/country/{country_str}/indicator/{indicator_code}"
    params = {
        "format": "json",
        "date": "2019:2023",
        "per_page": 2000
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data_json = response.json()
        data = pd.json_normalize(data_json[1])
        df = data[["country.id", "country.value", "date", "value"]].dropna()
        df.columns = ["country_code", "country", "year", indicator_name]
        df["year"] = df["year"].astype(int)
        return df
    else:
        print(f"Failed to fetch data for {indicator_code}")
        return pd.DataFrame()

df_pv = fetch_indicator_data("PV.EST", "political_stability")
df_cc = fetch_indicator_data("CC.EST", "control_of_corruption")

df_combined = pd.merge(df_pv, df_cc, on=["country", "year"], how="outer")

df_combined = df_combined.sort_values(["country", "year"])
print(df_combined.head(10))

df_combined.to_csv("wgi_political_corruption_2019_2023.csv", index=False)

  country_code_x     country  year  political_stability country_code_y  \
0             AR   Argentina  2019            -0.097823             AR   
1             AR   Argentina  2020            -0.071997             AR   
2             AR   Argentina  2021             0.000530             AR   
3             AR   Argentina  2022            -0.098594             AR   
4             AR   Argentina  2023            -0.127518             AR   
5             AZ  Azerbaijan  2019            -0.693411             AZ   
6             AZ  Azerbaijan  2020            -0.852254             AZ   
7             AZ  Azerbaijan  2021            -0.837824             AZ   
8             AZ  Azerbaijan  2022            -0.922807             AZ   
9             AZ  Azerbaijan  2023            -0.730754             AZ   

   control_of_corruption  
0              -0.107116  
1              -0.163570  
2              -0.422597  
3              -0.447222  
4              -0.360884  
5              -0.84968

In [10]:
def fetch_gci(start_year=2019, end_year=2023):
    country_str = ";".join(countries)
    url = f"https://api.worldbank.org/v2/country/{country_str}/indicator/GCI.INDEX.XQ"
    params = {
        "format": "json",
        "date": f"{start_year}:{end_year}",
        "per_page": 5000
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    records = data[1] if len(data) > 1 else []
    df = pd.json_normalize(records)
    df = df[["country.id", "country.value", "date", "value"]].dropna()
    df.columns = ["country_code", "country", "year", "gci"]
    df["year"] = df["year"].astype(int)
    return df.sort_values(["country", "year"])

if __name__ == "__main__":
    df_gci = fetch_gci()
    print(df_gci.head(10))
    df_gci.to_csv("gci_39pays_2019_2023.csv", index=False)

   country_code           country  year   gci
4           CIV     Cote d'Ivoire  2008  3.50
3           CIV     Cote d'Ivoire  2009  3.43
2           CIV     Cote d'Ivoire  2010  3.35
1           CIV     Cote d'Ivoire  2011  3.37
59          EGY  Egypt, Arab Rep.  2006  4.00
58          EGY  Egypt, Arab Rep.  2007  3.96
57          EGY  Egypt, Arab Rep.  2008  4.00
56          EGY  Egypt, Arab Rep.  2009  4.04
55          EGY  Egypt, Arab Rep.  2010  4.00
54          EGY  Egypt, Arab Rep.  2011  3.88


In [11]:
from google.colab import files
df_combined.to_csv("wgi_political_corruption_2019_2023.csv", index=False)
df_gci.to_csv("gci_39pays_2019_2023.csv", index=False)
files.download("wgi_political_corruption_2019_2023.csv")
files.download("gci_39pays_2019_2023.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Charger le fichier
df = pd.read_excel("Final_Data_Formatted_Panel.xlsx")

# Créer une variable log du PIB
df['Log_GDP'] = np.log(df['GDP (constant 2015 US$)'])

# Définir les variables
y = df['Foreign direct investment, net inflows (% of GDP)']
X = df[[
    'GDP growth (annual %)',
    'Log_GDP',
    'Inflation, consumer prices (annual %)',
    'Trade (% of GDP)',
    'Political stability',
    'Control of corruption',
    'Individuals using the Internet (% of population)'
]]

# Supprimer les lignes incomplètes
df_model = pd.concat([y, X], axis=1).dropna()
y_clean = df_model['Foreign direct investment, net inflows (% of GDP)']
X_clean = df_model.drop(columns='Foreign direct investment, net inflows (% of GDP)')
X_clean = sm.add_constant(X_clean)

# Estimer la régression
model = sm.OLS(y_clean, X_clean).fit()

# Afficher les résultats
print(model.summary())

# Visualiser les coefficients
coefs = model.params
conf = model.conf_int()
errors = (conf[1] - conf[0]) / 2

plt.figure(figsize=(10, 6))
sns.barplot(x=coefs.values, y=coefs.index, orient='h', xerr=errors.values, palette='crest')
plt.title("OLS Coefficients with 95% Confidence Intervals")
plt.xlabel("Coefficient")
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'Final_Data_Formatted_Panel.xlsx'